# AWS S3 com PySpark

Este notebook cobre:
- Configuração do Spark para S3
- Leitura e escrita de dados no S3
- Boas práticas e otimizações
- Formatos de arquivo recomendados

## 1. Configuração do Spark para S3

In [ ]:
from pyspark.sql import SparkSession

# Configuração básica para S3
spark = SparkSession.builder \
    .appName("SparkS3") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.access.key", "YOUR_ACCESS_KEY") \
    .config("spark.hadoop.fs.s3a.secret.key", "YOUR_SECRET_KEY") \
    .config("spark.hadoop.fs.s3a.endpoint", "s3.amazonaws.com") \
    .getOrCreate()

In [ ]:
# Usando IAM Role (recomendado em EMR/EC2)
spark_iam = SparkSession.builder \
    .appName("SparkS3IAM") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", 
            "com.amazonaws.auth.InstanceProfileCredentialsProvider") \
    .getOrCreate()

In [ ]:
# Usando profile do AWS CLI
spark_profile = SparkSession.builder \
    .appName("SparkS3Profile") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", 
            "com.amazonaws.auth.profile.ProfileCredentialsProvider") \
    .getOrCreate()

## 2. Configurações de Performance

In [ ]:
# Spark otimizado para S3
spark_optimized = SparkSession.builder \
    .appName("SparkS3Optimized") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.access.key", "YOUR_ACCESS_KEY") \
    .config("spark.hadoop.fs.s3a.secret.key", "YOUR_SECRET_KEY") \
    \
    .config("spark.hadoop.fs.s3a.connection.maximum", "100") \
    .config("spark.hadoop.fs.s3a.fast.upload", "true") \
    .config("spark.hadoop.fs.s3a.fast.upload.buffer", "bytebuffer") \
    .config("spark.hadoop.fs.s3a.multipart.size", "104857600") \
    .config("spark.hadoop.fs.s3a.block.size", "134217728") \
    \
    .config("spark.sql.parquet.mergeSchema", "false") \
    .config("spark.sql.parquet.filterPushdown", "true") \
    .config("spark.hadoop.parquet.enable.summary-metadata", "false") \
    \
    .config("spark.speculation", "false") \
    .getOrCreate()

## 3. Lendo Dados do S3

In [ ]:
# Lendo Parquet
df_parquet = spark.read.parquet("s3a://my-bucket/data/parquet/")
df_parquet.show()

# Lendo particionado
df_partitioned = spark.read.parquet("s3a://my-bucket/data/partitioned/")
df_partitioned.printSchema()

In [ ]:
# Lendo CSV
df_csv = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("s3a://my-bucket/data/csv/")

# Lendo JSON
df_json = spark.read.json("s3a://my-bucket/data/json/")

In [ ]:
# Lendo com glob pattern
df_multi = spark.read.parquet("s3a://my-bucket/data/year=2024/month=*/")

# Lendo múltiplos paths
paths = [
    "s3a://my-bucket/data/region=us/",
    "s3a://my-bucket/data/region=eu/"
]
df_regions = spark.read.parquet(*paths)

## 4. Escrevendo Dados no S3

In [ ]:
# Dados de exemplo
data = [
    (1, "João", "Vendas", 5000.0, "2024-01-15"),
    (2, "Maria", "TI", 7500.0, "2024-01-20"),
    (3, "Pedro", "Vendas", 4500.0, "2024-02-10"),
]
df = spark.createDataFrame(data, ["id", "nome", "departamento", "salario", "data"])

In [ ]:
# Escrevendo Parquet
df.write \
    .mode("overwrite") \
    .parquet("s3a://my-bucket/output/parquet/")

In [ ]:
# Escrevendo particionado
df.write \
    .mode("overwrite") \
    .partitionBy("departamento") \
    .parquet("s3a://my-bucket/output/partitioned/")

In [ ]:
# Escrevendo com compressão específica
df.write \
    .mode("overwrite") \
    .option("compression", "snappy") \
    .parquet("s3a://my-bucket/output/compressed/")

In [ ]:
# Controlando número de arquivos
df.coalesce(1) \
    .write \
    .mode("overwrite") \
    .parquet("s3a://my-bucket/output/single-file/")

# Ou usando repartition para distribuir melhor
df.repartition(4) \
    .write \
    .mode("overwrite") \
    .parquet("s3a://my-bucket/output/four-files/")

## 5. Dynamic Partition Overwrite

In [ ]:
# Configurar modo dinâmico
spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")

# Agora só sobrescreve partições presentes no DataFrame
df_vendas = df.filter(df.departamento == "Vendas")
df_vendas.write \
    .mode("overwrite") \
    .partitionBy("departamento") \
    .parquet("s3a://my-bucket/output/partitioned/")

## 6. S3 Select (Pushdown)

In [ ]:
# S3 Select permite filtrar dados no S3 antes de transferir
# Requer formato CSV ou JSON no S3

# Configurar S3 Select
df_s3select = spark.read \
    .format("s3selectCSV") \
    .option("header", "true") \
    .option("delimiter", ",") \
    .load("s3a://my-bucket/data/large.csv")

# Filtro será pushdown para S3
df_filtered = df_s3select.filter(df_s3select.salario > 5000)

## 7. Boas Práticas S3 + Spark

### Estrutura de Diretórios:
```
s3://bucket/
├── raw/                    # Dados brutos
│   └── source/
│       └── year=2024/
│           └── month=01/
├── processed/              # Dados processados
│   └── domain/
│       └── table/
├── curated/                # Dados curados
└── temp/                   # Arquivos temporários
```

### Dicas:
1. Use Parquet ou ORC (nunca CSV para analytics)
2. Particione por colunas frequentemente filtradas
3. Evite arquivos muito pequenos (< 128MB)
4. Use compressão Snappy ou ZSTD
5. Prefira s3a:// ao invés de s3://

In [ ]:
# Exemplo de pipeline completo
def process_s3_data(input_path, output_path):
    # Ler dados
    df = spark.read.parquet(input_path)
    
    # Transformar
    df_processed = df \
        .filter(df.valor > 0) \
        .withColumn("ano", F.year("data")) \
        .withColumn("mes", F.month("data"))
    
    # Escrever particionado
    df_processed \
        .repartition("ano", "mes") \
        .write \
        .mode("overwrite") \
        .partitionBy("ano", "mes") \
        .option("compression", "snappy") \
        .parquet(output_path)

# process_s3_data(
#     "s3a://my-bucket/raw/vendas/",
#     "s3a://my-bucket/processed/vendas/"
# )